# extract_images.ipynb

Run this notebook using the **Python 3** kernel.

**pa1.ijm**
```
run("8-bit");
run("Bandpass Filter...", "filter_large=40 filter_small=3 suppress=None tolerance=5 autoscale saturate");
setAutoThreshold("Triangle");
run("Analyze Particles...", "size=1000-Infinity circularity=[] display exclude");
saveAs("Results", "/home/aubrey/Desktop/bug_soup/Results.csv");
eval("script", "System.exit(0);");
```

**Command line:**
```
/home/aubrey/Fiji.app/ImageJ-linux64 P5150004.JPG -macro pa.ijm
```


# References

https://www.youtube.com/watch?v=ayQSqdOPR-c

In [13]:
import pandas as pd
import cv2
import os
import shutil
import numpy as np

In [14]:
RESULTS_PATH = '/home/aubrey/Desktop/bug_soup/Results.csv'
OUTPUT_DIR = 'insect_images'
IMAGEJ_PATH = '/home/aubrey/Fiji.app/ImageJ-linux64'
IMAGE_PATH = '/home/aubrey/Desktop/bug_soup/P5150004.JPG'
#PARTICLE_ANALYSIS_RESULTS_FILE = 'P5150004.csv'
#IMAGE_DIR = 'P5150004'
MARGIN = 10

In [15]:
os.path.basename(IMAGE_PATH).split('.')[0]

'P5150004'

In [16]:
# Create OUTPUT_DIR if it does not already exist

if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

# Create or recreate a folder for the image in OUTPUT_DIR

image_file_name = os.path.basename(IMAGE_PATH)
image_output_dir = f'{OUTPUT_DIR}/{image_file_name.split(".")[0]}'
shutil.rmtree(image_output_dir, ignore_errors=True)
os.mkdir(image_output_dir)

# Copy image into the new folder

os.system(f'cp {IMAGE_PATH} {image_output_dir}/{image_file_name}')  

# Perform particle analysis using ImageJ

image_path = f'{image_output_dir}/{image_file_name}'
command_line = f'{IMAGEJ_PATH} {image_path} -macro pa1.ijm'
print(command_line)
os.system(command_line)

/home/aubrey/Fiji.app/ImageJ-linux64 insect_images/P5150004/P5150004.JPG -macro pa1.ijm


0

In [17]:
df = pd.read_csv(RESULTS_PATH)
df.rename(columns={' ': 'i'}, inplace=True)
df.sort_values('Area', inplace=True)
df['i'] = np.arange(1,df.shape[0]+1)
csv_path = f'{image_path.split(".")[0]}.csv'
df.to_csv(csv_path, index=False)
df

,i,Area,Mean,StdDev,Min,Max,X,Y,Perim.,BX,...,Angle,Circ.,Feret,FeretX,FeretY,FeretAngle,MinFeret,AR,Round,Solidity
9,1,1030,123.458,21.112,17,139,1400.662,665.636,258.610,1341,...,0.695,0.194,111.759,1341,662,173.320,19.665,9.050,0.110,0.596
4,2,1037,96.916,29.526,25,139,812.977,360.838,153.924,783,...,139.660,0.550,67.476,783,338,140.412,26.163,2.559,0.391,0.904
53,3,1069,83.788,39.589,0,139,522.825,2444.478,189.321,493,...,131.620,0.375,84.959,495,2418,132.138,18.605,5.491,0.182,0.855
34,4,1076,97.189,29.037,26,139,2049.836,1853.964,221.664,2006,...,31.233,0.275,97.417,2007,1880,31.569,16.861,7.565,0.132,0.750
12,5,1091,40.786,48.522,0,139,2790.598,844.787,143.196,2767,...,88.902,0.669,46.271,2787,819,96.203,37.691,1.212,0.825,0.874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,69,26367,73.492,42.650,0,139,1720.891,2676.140,2532.172,1561,...,177.337,0.052,334.073,1564,2655,158.581,216.094,1.969,0.508,0.534
58,70,26699,127.515,8.699,71,139,3570.434,2815.445,5031.294,3211,...,24.807,0.013,845.592,3218,2976,24.898,84.568,17.184,0.058,0.461
67,71,27228,127.091,10.204,7,139,1446.118,2975.646,4542.871,846,...,0.279,0.017,1205.034,846,2994,0.428,51.525,39.792,0.025,0.494
18,72,29158,68.025,46.251,0,139,2195.344,1314.756,1859.489,2012,...,39.717,0.106,401.368,2014,1432,38.526,186.701,2.225,0.449,0.590


In [18]:
# Use bounding box values to extract images.
# Images with bounding boxes plus margin which exceed the original image edges are excluded.

image = cv2.imread(image_path)
ymax = image.shape[0] - 1
xmax = image.shape[1] - 1

for row in df.itertuples():
    y1 = row.BY - MARGIN
    y2 = row.BY + row.Height + MARGIN
    x1 = row.BX - MARGIN
    x2 = row.BX + row.Width + MARGIN
    if (y1 >= 0) and (y2 <= ymax) and (x1 >= 0) and (x2 <= xmax):
        roi = image[y1:y2, x1:x2]
        filename = f'{image_output_dir}/{row.i:03}.jpg'
        cv2.imwrite(filename, roi)